In [6]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)

def has_link(content):
    if content == None or content == "NULL":
        return False
    
    return "http://" in content or "https://" in content

def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()


def comment_has_link(user_id):
    comments = query('''select comment.content
    from comment
    join user
    on user.id = comment.author_id''' )

    for user in user_comments:
       if user[0] == user_id and content_has_link(user[1]):
            return user[1]
    return False
    
    
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df_potential_bot = df[df["edits"] + df["reviews"] + df["taxonomy"] == 0].copy()
user_ids = ",".join(df_potential_bot["id"].map(str))

df = pd.read_sql(f"""
    select user.id, user.username, comment.content as comment
    from user
    join comment on user.id = comment.author_id
    where user.id in ({user_ids})
""", db)
df = df[df["comment"].map(has_link)].copy()
df.to_csv("/tmp/2022-21-03users_with_link_in_comment.csv")
df



,id,username,comment
1,38724,kraeml,"Ok, danke fÃ¼r die Einladung. Werde mir die Ta..."
11,41334,anjanajain727,\r\nNon commercial relocation is definitely an...
12,41334,anjanajain727,\r\nTransferring household! Housing moving! Sw...
15,41334,anjanajain727,\r\nNon commercial move can be an high-priced ...
16,41334,anjanajain727,\r\n\r\nRelocating residence! Residential move...
...,...,...,...
1657,223991,nikkikaur,https://foro.zendalibros.com/forums/users/nikk...
1658,223991,nikkikaur,https://www.bustyescorts.net/mumbai-russian-es...
1690,226892,neeivamar23,If you havenâ€™t tried Taco Bell Breakfast yet...
1776,236675,GwPatronHans,https://pornhub.com
